In [1]:
from wmin.api import API

from super_net.covmats import sqrt_covmat_jax

import jax.scipy.linalg as jla
import jax
import jax.numpy as jnp

import time
import timeit, functools

In [350]:
inp = {
    "dataset_inputs": [
{'dataset':'NMC'},
{'dataset': 'SLACP_dwsh'},
{'dataset': 'SLACD_dw_ite'},
{'dataset': 'BCDMSP_dwsh'},
{'dataset': 'BCDMSD_dw_ite'},
{'dataset': 'CHORUSNUPb_dw_ite'},
{'dataset': 'CHORUSNBPb_dw_ite'},
{'dataset': 'NTVNUDMNFe_dw_ite', 'cfac': ['MAS']},
{'dataset': 'NTVNBDMNFe_dw_ite', 'cfac': ['MAS']},
{'dataset': 'HERACOMBNCEM'} ,
{'dataset': 'HERACOMBNCEP575'},
{'dataset': 'HERACOMBNCEP820'},
{'dataset': 'HERACOMBNCEP920'},
{'dataset': 'HERACOMBNCEP460'},
{'dataset': 'HERACOMBCCEP'},
{'dataset': 'HERACOMBCCEM'} ,
{'dataset':'HERACOMB_SIGMARED_B'},
{'dataset': 'HERACOMB_SIGMARED_C'}
                      ],

  #   "positivity":{
  # "posdatasets":[
  # {"dataset": "POSF2U", "maxlambda": 1e6},      # Positivity Lagrange Multiplier
  # # - {dataset: POSF2DW, maxlambda: 1e6}
  # # - {dataset: POSF2S, maxlambda: 1e6}
  # # - {dataset: POSFLL, maxlambda: 1e6}
  # # - {dataset: POSF2C, maxlambda: 1e6}
  # # - {dataset: POSXUQ, maxlambda: 1e6}        # Positivity of MSbar PDFs
  # # - {dataset: POSXUB, maxlambda: 1e6}
  # # - {dataset: POSXDQ, maxlambda: 1e6}
  # # - {dataset: POSXDB, maxlambda: 1e6}
  # # - {dataset: POSXSQ, maxlambda: 1e6}
  # # - {dataset: POSXSB, maxlambda: 1e6}
  # # - {dataset: POSXGL, maxlambda: 1e6}
  # # - {dataset: POSDYU, maxlambda: 1e10}      
  # # - {dataset: POSDYD, maxlambda: 1e10}
  # # - {dataset: POSDYS, maxlambda: 1e10}
  #   ]
  #   },
    
    "theoryid": 400,
    "use_cuts": "internal",
    
    # wmin basis specs
    "wminpdfset": "210623_mnc_disonly_linear_1000",
    "n_replicas_wmin": 100,
    
    # Level 0 closure test
    # "fakedata": True,
    # "pseudodata": False, 
    # "closure_test_pdf": "210623_mnc_disonly_linear_1000",
    
    # fit specs
    "use_t0": True,
    "t0pdfset": "210623_mnc_disonly_linear_1000",
    
    # "bayesian_fit": True,
    "wmin_grid_index": 1, # random seed used for random parametrisation of wmin pdf
#     "replica_index": 1,  # random seed used for random noise to central values
    "trval_index": 1
}

In [3]:
weight_minimization_grid = API.weight_minimization_grid(**inp)
data_values = API.make_data_values(**inp)
pred_data = API.make_pred_data(**inp)

LHAPDF 6.5.0 loading all 985 PDFs in set 210623_mnc_disonly_linear_1000
210623_mnc_disonly_linear_1000, version 1; 985 PDF members


2023-11-20 20:10:06.133193: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:06.298871: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:06.323397: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:06.505430: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:06.634335: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:06.635018: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


LHAPDF 6.5.0 loading /Users/luca/opt/miniconda3/envs/supernet/share/LHAPDF/210623_mnc_disonly_linear_1000/210623_mnc_disonly_linear_1000_0000.dat
210623_mnc_disonly_linear_1000 PDF set, member #0, version 1


2023-11-20 20:10:22.246905: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2023-11-20 20:10:22.361095: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


In [288]:
def make_chi2_original(
    make_data_values,
    make_pred_data,
):
    """
    Returns a jax.jit compiled function that computes the chi2
    of a pdf grid on a dataset.

    Notes:
        - This function is designed for Bayesian like PDF fits.

    Parameters
    ----------
    make_data_values: training_validation.MakeDataValues
        dataclass containing data for training and validation.

    make_pred_data: theory_predictions.make_pred_data
        super_net provider for (fktable) theory predictions.

    make_posdata_split: training_validation.PosdataTrainValidationSplit
        dataclass inheriting from monte_carlo_utils.TrainValidationSplit

    make_penalty_posdata: theory_predictions.make_penalty_posdata
        super_net provider used to compute positivity penalty.

    alpha: float

    lambda_positivity: float

    Returns
    -------
    @jax.jit Callable
        function to compute chi2 of a pdf grid.

    """
    training_data = make_data_values.training_data
    central_values = training_data.central_values
    covmat = training_data.covmat
    central_values_idx = training_data.central_values_idx

    # sqrt_covmat = jnp.array(sqrt_covmat_jax(covmat))
    inv_covmat = jla.inv(covmat)

    @jax.jit
    def chi2(pdf):
        """ """
        diff = make_pred_data(pdf)[central_values_idx] - central_values

        # sqrt_covmat = jnp.array(sqrt_covmat_jax(covmat))

        # solve_triangular: solve the equation a x = b for x, assuming a is a triangular matrix.
        # chi2_vec = jla.solve_triangular(sqrt_covmat, diff, lower=True)
        # loss = jnp.sum(chi2_vec**2)
        loss = jnp.einsum("i,ij,j", diff, inv_covmat, diff)

        return loss

    return chi2

chi2_original = make_chi2_original(data_values, pred_data)

In [289]:
@jax.jit
def log_likelihood(weights):
    """
    TODO
    """
    wmin_weights = jnp.concatenate((jnp.array([1.0]), weights))
    pdf = jnp.einsum(
        "i,ijk", wmin_weights, weight_minimization_grid.wmin_INPUT_GRID
    )
    return -0.5 * chi2_original(pdf)

In [239]:
def make_chi2_opt(make_data_values, make_pred_data):
    """
    TODO
    """
    training_data = make_data_values.training_data
    central_values = training_data.central_values
    covmat = training_data.covmat
    central_values_idx = training_data.central_values_idx

    predictions = []
    for i in range(weight_minimization_grid.wmin_INPUT_GRID.shape[0]):
        predictions.append(make_pred_data(weight_minimization_grid.wmin_INPUT_GRID[i]))
    predictions = jnp.array(predictions)

    sqrt_covmat = jnp.array(sqrt_covmat_jax(covmat))

    inv_covmat = jla.inv(covmat)

    @jax.jit
    def chi2(weights):
        """
        TODO
        """
        wmin_weights = jnp.concatenate((jnp.array([1.0]), weights))
        theory = jnp.einsum(
            "i,ij", wmin_weights, predictions
        )
        diff = (
            theory[central_values_idx]
            - central_values
        )

        # solve_triangular: solve the equation a x = b for x, assuming a is a triangular matrix.
        # chi2_vec = jla.solve_triangular(sqrt_covmat, diff, lower=True)
        # loss = jnp.sum(chi2_vec**2)

        loss = jnp.einsum("i,ij,j", diff, inv_covmat, diff)
        
        return loss

    return chi2

chi2_opt = make_chi2_opt(data_values, pred_data)

In [240]:
@jax.jit
def log_likelihood_opt(weights):
    """
    TODO
    """
    return -0.5 * chi2_opt(weights)

In [325]:
weights = jax.random.uniform(jax.random.PRNGKey(758493), shape=(100, 99))

In [319]:
chi2s_or = []
t0 = time.time()
for weight in weights:
    chi2s_or.append(log_likelihood(weight))

t1 = time.time()

total = t1-t0

print("Time for evalutation:", total)

Time for evalutation: 8.086869955062866


In [321]:
chi2s_opt = []
t0 = time.time()
for weight in weights:
    chi2s_opt.append(log_likelihood_opt(weight))

t1 = time.time()

total = t1-t0

print("Time for evalutation:", total)

Time for evalutation: 6.619807958602905
